In [1]:
from fpl_loader import FplLoader
from team import Team, optimize_team
from player import Player
from expected_points_calculator import ExpectedPointsCalculator
import heapq

### Who is playing in GW 37?

In [2]:
FplLoader.get_fixtures_for_gameweek(37)

for i in range(1, 21):
    fs = []
    for f in FplLoader.get_fixtures_for_gameweek(37):
        if f['team_h'] == i:
            fs.append(FplLoader.get_team_basic_info(f['team_a'])['name'])
        if f['team_a'] == i:
            fs.append(FplLoader.get_team_basic_info(f['team_h'])['name'])
            
    if len(fs)==2:
        print("Team {} play against {} ".format(FplLoader.get_team_basic_info(i)['name'], fs))


Team Brighton play against ['Newcastle', 'Chelsea'] 
Team Chelsea play against ["Nott'm Forest", 'Brighton'] 
Team Man City play against ['Fulham', 'Spurs'] 
Team Man Utd play against ['Arsenal', 'Newcastle'] 
Team Newcastle play against ['Brighton', 'Man Utd'] 
Team Spurs play against ['Burnley', 'Man City'] 


### Downloading my team

In [3]:
player_list = []
picks = FplLoader.get_my_team_from_api('aledvaghela@gmail.com', 'NCP@jxy7pau-bvm3cuf', 2085446)['picks']
for pick in picks:
    d = {'element': pick['element'],
         'name': FplLoader.get_player_basic_info(pick['element'])['web_name'],
         'position': FplLoader.get_player_basic_info(pick['element'])['element_type'],
         'club': FplLoader.get_player_basic_info(pick['element'])['team'],
         'cost': pick['selling_price'],
        }
    player_list.append(Player(**d))

money_in_bank = FplLoader.get_my_team_from_api('aledvaghela@gmail.com', 'NCP@jxy7pau-bvm3cuf', 2085446)['transfers']['bank']
free_transfers = FplLoader.get_my_team_from_api('aledvaghela@gmail.com', 'NCP@jxy7pau-bvm3cuf', 2085446)['transfers']['limit']
team = Team(player_list, money_in_bank, free_transfers)
print(team)
print("Team is feasible: ", team.is_feasible())

---------------------------------------------------------------------------
GKP
Player(element=263, name='Pickford', position=1, club=9, cost=47)
Player(element=687, name='Petrović', position=1, club=7, cost=46)
---------------------------------------------------------------------------
DEF
Player(element=407, name='Burn', position=2, club=15, cost=46)
Player(element=616, name='Gvardiol', position=2, club=13, cost=50)
Player(element=506, name='Pedro Porro', position=2, club=18, cost=58)
Player(element=150, name='Van Hecke', position=2, club=5, cost=40)
Player(element=5, name='Gabriel', position=2, club=1, cost=52)
---------------------------------------------------------------------------
MID
Player(element=353, name='Foden', position=3, club=13, cost=83)
Player(element=412, name='Gordon', position=3, club=15, cost=62)
Player(element=516, name='Son', position=3, club=18, cost=98)
Player(element=362, name='Palmer', position=3, club=7, cost=58)
Player(element=373, name='B.Fernandes', pos

### A method to calculate the expected points of an individual in a particular week

In [4]:
class ComplexExpectedPointsCalculator(ExpectedPointsCalculator):
    something_to_play_for = set([1, 2, 18, 13, 6, 12, 16])
    minutes_multiplier = {
        407: 1.00, # burn
        150: 0.00, # van hecke
        5: 1.00, # gabriel
        616: 0.80, #gvardiol
        506: 0.95, #porro
        
        29: 0.95, #white
        
        362: 1.00, #palmer
        373: 0.60, #fernandes
        353: 0.80, #foden
        516: 0.80, #son
        412: 0.90, #gordon
        
        349: 0.70, #de bruyne
        382: 0.75, #garnacho
        
        355: 0.80, #haaland
        60: 0.95, #watkins
        211: 0.95, # jackson
        
        415: 0.75, #isak
    }
    def get_expected_points(self, player_id: int, gameweek: int, as_of_gameweek: int) -> float:
        result = 0
        points_per_game = float(FplLoader.get_player_basic_info(player_id)['points_per_game'])
        form = float(FplLoader.get_player_basic_info(player_id)['form'])
        for fixture in FplLoader.get_player_future_info_for_gameweek(player_id, gameweek):
            our_team, opponent = -1, -1
            if fixture['is_home']:
                our_team = fixture['team_h']
                opponent = fixture['team_a']
            else:
                our_team = fixture['team_a']
                opponent = fixture['team_h']
            
            difficulty_alpha = 0
            if fixture['difficulty'] == 3:
                difficulty_alpha = 0
            elif fixture['difficulty'] < 3:
                difficulty_alpha = 0.5
            else:
                difficulty_alpha = -0.5
                
            on_the_beach_alpha = 0
            if our_team in ComplexExpectedPointsCalculator.something_to_play_for:
                on_the_beach_alpha += 0.5
            else:
                on_the_beach_alpha -= 0.5
                
            if opponent in ComplexExpectedPointsCalculator.something_to_play_for:
                on_the_beach_alpha -= 0.5
            else:
                on_the_beach_alpha += 0.5
                
            result += 0.5 * (points_per_game + form) + difficulty_alpha + on_the_beach_alpha
            
        return result * ComplexExpectedPointsCalculator.minutes_multiplier.get(player_id, 1.0)

### Getting the optimal team and captain for a particular gameweek

In [6]:
cepc = ComplexExpectedPointsCalculator()

for i in range(1, 5):
    for p in team.players[i]:
        print('{}, {}'.format(FplLoader.get_player_basic_info(p.element)['web_name'], cepc.get_expected_points(p.element, 37, 36)) )

team.optimal_formation(cepc, 37, 36)

Pickford, 5.6
Petrović, 7.699999999999999
Burn, 5.9
Gvardiol, 11.280000000000001
Pedro Porro, 3.8
Van Hecke, 0.0
Gabriel, 4.55
Foden, 9.680000000000001
Gordon, 12.420000000000002
Son, 8.4
Palmer, 15.100000000000001
B.Fernandes, 9.45
Watkins, 5.8425
Haaland, 13.280000000000001
N.Jackson, 11.114999999999998


{1: {(Player(element=687, name='Petrović', position=1, club=7, cost=46),
   7.699999999999999)},
 2: {(Player(element=5, name='Gabriel', position=2, club=1, cost=52), 4.55),
  (Player(element=407, name='Burn', position=2, club=15, cost=46), 5.9),
  (Player(element=616, name='Gvardiol', position=2, club=13, cost=50),
   11.280000000000001)},
 3: {(Player(element=353, name='Foden', position=3, club=13, cost=83),
   9.680000000000001),
  (Player(element=362, name='Palmer', position=3, club=7, cost=58),
   15.100000000000001),
  (Player(element=373, name='B.Fernandes', position=3, club=14, cost=84),
   9.45),
  (Player(element=412, name='Gordon', position=3, club=15, cost=62),
   12.420000000000002),
  (Player(element=516, name='Son', position=3, club=18, cost=98), 8.4)},
 4: {(Player(element=211, name='N.Jackson', position=4, club=7, cost=69),
   11.114999999999998),
  (Player(element=355, name='Haaland', position=4, club=13, cost=141),
   13.280000000000001)}}

### Finding the optimal transfers over a particular horizon

In [8]:
candidate_list_web_names = ['Isak',
                            'White',
                            'De Bruyne',
                            'Garnacho',
                            'Romero',
                            'Van de Ven'
                           ]
candidate_list = []

for p in FplLoader.get_static_info()['elements']:
    if p['web_name'] in candidate_list_web_names:
        d = {
            'element': p['id'],
            'name': p['web_name'],
            'position': p['element_type'],
            'club':  p['team'],
            'cost': p['now_cost']
        }
        candidate_list.append(Player(**d))
        
print(candidate_list)

[Player(element=29, name='White', position=2, club=1, cost=61), Player(element=349, name='De Bruyne', position=3, club=13, cost=106), Player(element=382, name='Garnacho', position=3, club=14, cost=50), Player(element=415, name='Isak', position=4, club=15, cost=84), Player(element=794, name='White', position=3, club=15, cost=45), Player(element=511, name='Romero', position=2, club=18, cost=51), Player(element=639, name='Van de Ven', position=2, club=18, cost=44)]


In [11]:
top_three = optimize_team(team, candidate_list, 3, ComplexExpectedPointsCalculator(), 36, 35, 2)
third_best_score, third_best_team = heapq.heappop(top_three)
second_best_score, second_best_team = heapq.heappop(top_three)
first_best_score, first_best_team = heapq.heappop(top_three)

In [12]:
print("Score: {}".format(first_best_score/4))
print(first_best_team)

Score: 31.55625
---------------------------------------------------------------------------
GKP
Player(element=263, name='Pickford', position=1, club=9, cost=47)
Player(element=687, name='Petrović', position=1, club=7, cost=46)
---------------------------------------------------------------------------
DEF
Player(element=407, name='Burn', position=2, club=15, cost=46)
Player(element=616, name='Gvardiol', position=2, club=13, cost=50)
Player(element=506, name='Pedro Porro', position=2, club=18, cost=58)
Player(element=150, name='Van Hecke', position=2, club=5, cost=40)
Player(element=5, name='Gabriel', position=2, club=1, cost=52)
---------------------------------------------------------------------------
MID
Player(element=353, name='Foden', position=3, club=13, cost=83)
Player(element=362, name='Palmer', position=3, club=7, cost=58)
Player(element=516, name='Son', position=3, club=18, cost=98)
Player(element=412, name='Gordon', position=3, club=15, cost=62)
Player(element=373, name='B

In [20]:
print("Score: {}".format(second_best_score/4))
print(second_best_team)

Score: 67.33875
---------------------------------------------------------------------------
GKP
Player(element=687, name='Petrović', position=1, club=7, cost=46)
Player(element=263, name='Pickford', position=1, club=9, cost=47)
---------------------------------------------------------------------------
DEF
Player(element=5, name='Gabriel', position=2, club=1, cost=52)
Player(element=407, name='Burn', position=2, club=15, cost=45)
Player(element=150, name='Van Hecke', position=2, club=5, cost=40)
Player(element=616, name='Gvardiol', position=2, club=13, cost=49)
Player(element=29, name='White', position=2, club=1, cost=61)
---------------------------------------------------------------------------
MID
Player(element=353, name='Foden', position=3, club=13, cost=83)
Player(element=412, name='Gordon', position=3, club=15, cost=62)
Player(element=373, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=362, name='Palmer', position=3, club=7, cost=58)
Player(element=516, name='S

In [29]:
def get_expected_points_until_end_of_season(element):
    s = 0
    for i in [35, 36, 37, 38]:
        s += cepc.get_expected_points(element, i, 34)
    print('{}, {}'.format(FplLoader.get_player_basic_info(element)['web_name'], s))
    

In [30]:
get_expected_points_until_end_of_season(415)
get_expected_points_until_end_of_season(427)
get_expected_points_until_end_of_season(60)
get_expected_points_until_end_of_season(407)

Isak, 23.449999999999996
Schär, 21.75
Watkins, 28.2
Burn, 16.5


In [33]:
print('{}, {}'.format(FplLoader.get_player_basic_info(373)['web_name'], cepc.get_expected_points(373, 35, 34) + cepc.get_expected_points(373, 36, 34) + cepc.get_expected_points(373, 37, 34)+cepc.get_expected_points(373, 38, 34)) )
print('{}, {}'.format(FplLoader.get_player_basic_info(349)['web_name'], cepc.get_expected_points(349, 35, 34) + cepc.get_expected_points(349, 36, 34) + cepc.get_expected_points(349, 37, 34) + cepc.get_expected_points(349, 38, 34)) )

B.Fernandes, 26.75
De Bruyne, 21.375000000000004


### What does the fixture difficulty score actually mean?

In [63]:
for fixture in FplLoader.get_player_detailed_info(362)['history']:
    fid = fixture['fixture']
    team_h = FplLoader.get_team_basic_info(FplLoader.get_fixture_info(fid)['team_h'])['name']
    team_a = FplLoader.get_team_basic_info(FplLoader.get_fixture_info(fid)['team_a'])['name']
    team_h_difficulty = FplLoader.get_fixture_info(fid)['team_h_difficulty'] # what is the difficulty home team plays against
    team_a_difficulty = FplLoader.get_fixture_info(fid)['team_a_difficulty'] # what is the difficulty away team plays against
    print("{} : {}, {} : {}".format(team_h, team_h_difficulty, team_a, team_a_difficulty))

Burnley : 5, Man City : 2
Man City : 3, Newcastle : 5
Sheffield Utd : 5, Man City : 2
Chelsea : 2, Nott'm Forest : 3
Bournemouth : 3, Chelsea : 2
Chelsea : 3, Aston Villa : 3
Fulham : 3, Chelsea : 3
Burnley : 3, Chelsea : 2
Chelsea : 5, Arsenal : 3
Chelsea : 2, Brentford : 3
Spurs : 3, Chelsea : 4
Chelsea : 5, Man City : 3
Newcastle : 3, Chelsea : 4
Chelsea : 2, Brighton : 3
Man Utd : 3, Chelsea : 4
Everton : 3, Chelsea : 2
Chelsea : 2, Sheffield Utd : 3
Wolves : 3, Chelsea : 2
Chelsea : 2, Crystal Palace : 3
Luton : 3, Chelsea : 2
Chelsea : 2, Fulham : 3
Liverpool : 3, Chelsea : 5
Chelsea : 2, Wolves : 3
Crystal Palace : 3, Chelsea : 2
Man City : 3, Chelsea : 5
Brentford : 3, Chelsea : 2
Chelsea : 3, Newcastle : 3
Chelsea : 2, Burnley : 3
Chelsea : 3, Man Utd : 3
Sheffield Utd : 3, Chelsea : 2
Chelsea : 2, Everton : 3
Arsenal : 3, Chelsea : 5


### How to find a particular player?

In [8]:
for p in FplLoader.get_static_info()['elements']:
    if p['web_name']=='White':
        print(p)
        
FplLoader.get_team_basic_info(FplLoader.get_player_basic_info(60)['team'])['name']

{'chance_of_playing_next_round': 100, 'chance_of_playing_this_round': 100, 'code': 198869, 'cost_change_event': 1, 'cost_change_event_fall': -1, 'cost_change_start': 6, 'cost_change_start_fall': -6, 'dreamteam_count': 5, 'element_type': 2, 'ep_next': '8.2', 'ep_this': '7.7', 'event_points': 1, 'first_name': 'Benjamin', 'form': '7.2', 'id': 29, 'in_dreamteam': True, 'news': '', 'news_added': '2023-11-12T02:00:08.673390Z', 'now_cost': 61, 'photo': '198869.jpg', 'points_per_game': '4.9', 'second_name': 'White', 'selected_by_percent': '22.1', 'special': False, 'squad_number': None, 'status': 'a', 'team': 1, 'team_code': 3, 'total_points': 168, 'transfers_in': 3004762, 'transfers_in_event': 133932, 'transfers_out': 1705800, 'transfers_out_event': 18437, 'value_form': '1.2', 'value_season': '27.5', 'web_name': 'White', 'minutes': 2739, 'goals_scored': 4, 'assists': 5, 'clean_sheets': 16, 'goals_conceded': 25, 'own_goals': 0, 'penalties_saved': 0, 'penalties_missed': 0, 'yellow_cards': 8, 're

'Aston Villa'

In [ ]:
DataLoader.get_position_info(3)

In [47]:
for p in FplLoader.get_static_info()['elements']:
    if float(p['points_per_game']) > 5:
        print(p['web_name'], p['points_per_game'])

Ødegaard 5.2
Saka 6.3
Watkins 6.6
Mbeumo 5.2
March 5.6
Palmer 7.4
Olise 6.3
Diogo J. 5.2
Salah 6.9
De Bruyne 5.2
Foden 6.1
Haaland 6.7
Gordon 5.3
Isak 5.5
Son 6.4
Bowen 5.3


In [5]:
for i in set([1, 2, 18, 13, 6, 12, 16]):
    print(i, FplLoader.get_team_basic_info(i)['name'])

1 Arsenal
2 Aston Villa
6 Burnley
12 Luton
13 Man City
16 Nott'm Forest
18 Spurs
